In [ ]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm

In [ ]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")

In [ ]:
train = pd.read_csv(os.path.join(path,"train_processed.csv"))
test = pd.read_csv(os.path.join(path,"test_processed.csv"))

In [ ]:
train_apt = pd.DataFrame(columns=['단지명','도로명주소','위도','경도'])
train_apt['단지명'] = train['단지명'].unique()
train_apt['도로명주소'] = train['도로명주소'].unique()

In [ ]:
test_apt = pd.DataFrame(columns=['단지명','도로명주소','위도','경도'])
test_apt['단지명'] = test['단지명'].unique()
test_apt['도로명주소'] = test['도로명주소'].unique()

In [ ]:
def get_location(array,dataframe):
    
    for i in tqdm(array):
        
        ind = dataframe[dataframe['도로명주소']==i].index
        url = "https://dapi.kakao.com/v2/local/search/keyword.json"

        queryString = {'query' : i}
        header = {'Authorization': 'KakaoAK 660c198b2093e22460c08a028177ac18'} #rest api

        response = requests.get(url, headers=header, params=queryString)
        tokens = response.json()
        
        try: 
            
            lat = tokens['documents'][0]['x']
            lng = tokens['documents'][0]['y']
            dataframe.loc[ind,'위도'] = lat
            dataframe.loc[ind,'경도'] = lng
            
        except:
            
            query = dataframe.loc[ind,'단지명']
            queryString = {'query' : query}
            header = {'Authorization': 'KakaoAK 660c198b2093e22460c08a028177ac18'} #rest api

            response = requests.get(url, headers=header, params=queryString)
            tokens = response.json()
            
            try:
                lat = tokens['documents'][0]['x']
                lng = tokens['documents'][0]['y']
                dataframe.iloc[ind,'위도'] = lat
                dataframe.iloc[ind,'경도'] = lng
            
            except:
                print(query)
                pass

In [ ]:
get_location(train['도로명주소'].unique(),train_apt)

In [ ]:
get_location(test['도로명주소'].unique(),test_apt)

In [ ]:
def get_location_na(original,name):
        
    ind = train_apt[train_apt['단지명']=='공주옥룡주공1차아파트'].index
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"

    queryString = {'query' : name}
    header = {'Authorization': 'KakaoAK 660c198b2093e22460c08a028177ac18'} #rest api

    response = requests.get(url, headers=header, params=queryString)
    tokens = response.json()
            
    lat = tokens['documents'][0]['x']
    lng = tokens['documents'][0]['y']
    train_apt.loc[ind,'위도'] = lat
    train_apt.loc[ind,'경도'] = lng

In [ ]:
train_apt[train_apt['위도'].isna()==True]

In [ ]:
test_apt[test_apt['위도'].isna()==True]

In [ ]:
get_location_na('공주옥룡주공1차아파트','옥룡주공1단지아파트')

In [ ]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")
train_apt.to_csv(os.path.join(path, "train_apt.csv"),index=False)
test_apt.to_csv(os.path.join(path, "test_apt.csv"),index=False)